# 실습형 문제

In [1]:
import sqlite3
conn = sqlite3.connect('data.db')
conn

In [2]:
# 커서 객체 반환. 커서는 SQL문 실행시키고 결과데이터(SELECT)를 조회하는데 사용객체
cursor = conn.cursor()
cursor

In [3]:
# 클래스 생성은 선택사항
class Member:
    '멤버 데이터 저장'
    def __init__(self, name, phone, email, age, grade, etc):
        self.name = name
        self.phone = phone
        self.email = email
        self.age = age
        self.grade = grade
        self.etc = etc
    def to_list_style(self): # 객체를 리스트 스타일로 ;"홍길동 010-8999-9999 e@e.com 20 3 까칠해"
        temp = [self.name, self.phone, self.email, str(self.age), str(self.grade), self.etc]
        return ' '.join(temp)
    def __str__(self): # *** 홍길동 010-8999-9999 e@e.com 20 3 까칠해
        return "{:>5}\t{:3}\t{:15}\t{:15}\t{:3}\t{}".format('*'*self.grade, # grade만큼 * 반복, 오른쪽 정렬로 5자리
                   self.name, self.phone, self.email, self.age, self.etc) 

In [4]:
def to_member(row):
    data = row.strip().split(' ')
    name = data[0]
    phone = data[1]
    email = data[2]
    age = int(data[3])
    grade = int(data[4])
    etc = data[5]
    return Member(name, phone, email, age, grade, etc)

In [5]:
m = to_member("홍길동 010-8999-9999 e@e.com 20 5 etc")
print(m)

*****	홍길동	010-8999-9999  	e@e.com        	 20	etc


In [6]:
# 1.입력
def insert_member():
    # 사용자로부터 이름, 전화, 이메일, 나이, 등급(1~5), 기타특징 입력받아 DB에 추가한다
    # commit
    name = input('이름 :')
    phone = input('전화 :')
    email = input('이메일 :')
    try:
        age = int(input('나이 :'))
        if age < 0 :
            age = 0
    except ValueError as e:
        print('유효하지 않은 나이를 입력하면 0으로 초기화')
        age = 0
    try:    
        grade = int(input('등급(1~5) :'))
        if grade < 1:
            grade = 1
        if grade > 5:
            grade = 5
    except ValueError as e:
        print('유효하지 않은 등급을 입력하면 1로 초기화')
        grade = 1
    etc = input('기타특징 :')
    cursor.execute("INSERT INTO MEMBER VALUES (?, ?, ?, ?, ?, ?) ", (name, phone, email, age, grade, etc))
    conn.commit()

In [7]:
# 위의 메소드 test
insert_member()

이름 :유길동
전화 :010-5555-5555
이메일 :y@a.com
나이 :21
등급(1~5) :2
기타특징 :etc


In [8]:
# 2.전체출력
def select_all():
    # "SELECT * FROM MEMBER"의 결과를 출력(DataFrame형태/ 리스트형태)
    import pandas as pd
    cursor.execute('SELECT * FROM MEMBER')
    member = cursor.fetchall() # 튜플리스트
    data_df = pd.DataFrame(member) # 데이터프레임
    data_df.columns = [row[0] for row in cursor.description]
    return data_df

In [9]:
# 위의 메소드 test
select_all()

,NAME,PHONE,EMAIL,AGE,GRADE,ETC
0,홍길동,010-9999-9999,h@a.com,23,4,etc
1,마길동,010-8888-8888,m@a.com,25,3,etc
2,김길동,010-7777-7777,k@a.com,31,2,etc
3,유길동,010-5555-5555,y@a.com,22,5,etc
4,서길동,010-6666-6666,s@a.com,22,5,etc
5,유길동,010-5555-5555,y@a.com,21,2,etc


In [10]:
# 3.이름검색
def select_name():
    # 사용자로부터 검색하고자 하는 이름을 받아 결과 출력(DataFrame형태/ 리스트형태)
    import pandas as pd
    name1 = input('검색할 이름 :')
    cursor.execute("SELECT * FROM MEMBER WHERE NAME = :name ", {'name':name1})
    member = cursor.fetchall() # 튜플리스트
    data_df = pd.DataFrame(member) # 데이터프레임
    data_df.columns = [row[0] for row in cursor.description]
    return data_df

In [11]:
# 위의 메소드 test
select_name()

검색할 이름 :유길동


,NAME,PHONE,EMAIL,AGE,GRADE,ETC
0,유길동,010-5555-5555,y@a.com,22,5,etc
1,유길동,010-5555-5555,y@a.com,21,2,etc


In [19]:
# 4.메일삭제
def delete_email():
    # 사용자로부터 삭제할 메일을 입력받아 해당 이메일이 있는지 확인하고
    # 해당 이메일이 없으면 없다 출력, 있으면 삭제하고 삭제했다 출력
    import pandas as pd
    email1 = input('삭제할 메일 :')
    cursor.execute("DELETE FROM MEMBER WHERE EMAIL = :email ", {'email':email1})
    member = cursor.fetchall() # 튜플리스트
    if member != [()]:
        print(email1, '을 삭제했습니다')
        conn.commit()
    else:
        print(email1, '님이 데이터에 존재하지 않습니다')

In [ ]:
def delete_email():
    # 사용자로부터 삭제할 메일을 입력받아 해당 이메일이 있는지 확인하고
    # 해당 이메일이 없으면 없다 출력, 있으면 삭제하고 삭제했다 출력
    email1 = input('삭제할 메일 :')
    delete_flag = False
    for i, member in enumerate(member):
        if member.email == email1:
            del member[i]
            delete_flag = True
            break
    if delete_flag:
        print(email1, '님을 삭제했습니다')
    else:
        print(email1, '님이 데이터에 존재하지 않습니다')

In [13]:
# 위의 메소드 test
delete_email()

삭제할 메일 :y@a.com
y@a.com 을 삭제했습니다


In [20]:
# 위의 메소드 test2
delete_email()

삭제할 메일 :a
a 을 삭제했습니다


In [21]:
# 지금 현재 DB 내용
cursor.execute("SELECT * FROM MEMBER")
cursor.fetchall()

[('홍길동', '010-9999-9999', 'h@a.com', 23, 4, 'etc'),
 ('마길동', '010-8888-8888', 'm@a.com', 25, 3, 'etc'),
 ('김길동', '010-7777-7777', 'k@a.com', 31, 2, 'etc'),
 ('서길동', '010-6666-6666', 's@a.com', 22, 5, 'etc')]

In [22]:
# 5.CSV내보내기
import csv
def save_csv():
    # "SELECT * FROM MEMBER"의 결과를 CSV로 내보내기(HEADER 포함)
    import pandas as pd
    cursor.execute('SELECT * FROM MEMBER')
    member = cursor.fetchall()
    data_df = pd.DataFrame(member)
    data_df
    with open('data.csv','w') as f :
        write = csv.writer(f)
        write.writerows(member)

In [23]:
# 위의 메소드 test
save_csv()

In [24]:
# 0.종료
def close_sql():
    global conn
    conn.close()
    print("종료합니다")

In [25]:
def main():
    while True:
        print("1:입력","2.전체조회","3:이름찾기","4.메일삭제","5:내보내기(CSV)","0.종료", sep=' | ', end='')
        fn = int(input('메뉴선택 : '))
        if fn == 1 :
            member = insert_member()
        elif fn == 2 :
            print(select_all())
        elif fn == 3 :
            print(select_name())
        elif fn == 4 :
            delete_email()
        elif fn == 5 :
            save_csv()
        elif fn == 0 :
            close_sql()
            break

In [26]:
if __name__=='__main__':
    import sqlite3
    global conn
    conn = sqlite3.connect('data.db')
    main()

1:입력 | 2.전체조회 | 3:이름찾기 | 4.메일삭제 | 5:내보내기(CSV) | 0.종료메뉴선택 : 1
이름 :이길동
전화 :010-3333-3333
이메일 :l@a.com
나이 :26
등급(1~5) :4
기타특징 :etc
1:입력 | 2.전체조회 | 3:이름찾기 | 4.메일삭제 | 5:내보내기(CSV) | 0.종료메뉴선택 : 2
  NAME          PHONE    EMAIL  AGE  GRADE  ETC
0  홍길동  010-9999-9999  h@a.com   23      4  etc
1  마길동  010-8888-8888  m@a.com   25      3  etc
2  김길동  010-7777-7777  k@a.com   31      2  etc
3  서길동  010-6666-6666  s@a.com   22      5  etc
4  이길동  010-3333-3333  l@a.com   26      4  etc
1:입력 | 2.전체조회 | 3:이름찾기 | 4.메일삭제 | 5:내보내기(CSV) | 0.종료메뉴선택 : 3
검색할 이름 :서길동
  NAME          PHONE    EMAIL  AGE  GRADE  ETC
0  서길동  010-6666-6666  s@a.com   22      5  etc
1:입력 | 2.전체조회 | 3:이름찾기 | 4.메일삭제 | 5:내보내기(CSV) | 0.종료메뉴선택 : 4
삭제할 메일 :l@a.com
l@a.com 을 삭제했습니다
1:입력 | 2.전체조회 | 3:이름찾기 | 4.메일삭제 | 5:내보내기(CSV) | 0.종료메뉴선택 : 2
  NAME          PHONE    EMAIL  AGE  GRADE  ETC
0  홍길동  010-9999-9999  h@a.com   23      4  etc
1  마길동  010-8888-8888  m@a.com   25      3  etc
2  김길동  010-7777-7777  k@a.com   31      2  etc
3  서길동 

# 서술형 문제

In [ ]:
# 1. 4번
# 2. 2번